In [1]:
import pandas as pd
import json
import re
import numpy as np

from sklearn.externals import joblib
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
pd.set_option('display.max_columns', 150)

In [78]:
sentences = pickle.load(open('cultivate_sentences.p', 'rb'))
messages = pickle.load(open('cultivate_messages.p', 'rb'))

In [80]:
sentences.head()

,sent_index,sentiment,thread_id,timestamp,sender_id,concrete,request_prob,polite_prob,question,msg_id,team_id,inform,conventional,request_action,request_info,msg_size,adv,articles,aux_verbs,convo_index,convo_size,imp_prons,female_recipients,female_prons,male_recipients,male_prons,subordinates,superiors,unique_tokens,gender_unknown,power_unknown,parent_predict,parent_prob,per_prons,preps,quants,response_time,sender_gender,sub_conjs
0,3,1,0,2001-10-05 02:38:26,0,3.980000,0.154175,0.447116,False,723973,0,0.269913,0.122803,0.005557,0.006111,3,0,0,0,0,1,0,0,0,5,0,0,0,21,6,11,0,NaN,0,0,0,NaN,I,0
1,2,2,0,2001-10-05 02:38:26,0,4.465000,0.229237,0.433185,False,723973,0,0.102239,0.345797,0.004580,0.009907,3,0,0,0,0,1,0,0,0,5,0,0,0,2,6,11,0,NaN,0,0,0,NaN,I,0
2,1,2,0,2001-10-05 02:38:26,0,2.160000,0.148573,0.399924,False,723973,0,0.309767,0.073297,0.006243,0.005516,3,0,0,0,0,1,0,0,0,5,0,0,0,4,6,11,0,NaN,0,0,0,NaN,I,0
3,0,1,0,2001-10-05 02:38:26,0,3.296667,0.047411,0.389760,False,723973,0,0.357122,0.000236,0.006312,0.002823,3,0,0,0,0,1,0,0,0,5,0,0,0,15,6,11,0,NaN,0,0,0,NaN,I,0
4,3,1,0,2001-10-06 07:38:26,0,3.980000,0.154175,0.447116,False,724694,0,0.269913,0.122803,0.005557,0.006111,3,0,0,0,0,1,0,0,0,5,0,0,0,21,6,11,0,NaN,0,0,0,NaN,I,0


In [44]:
def map_col_to_messages(col_name, df1 = sentences, df2 = messages):
    msg_ids = df1['msg_id'].tolist()
    col = df1[col_name].tolist()
    map_msg_thread = {}
    for msg, c in zip(msg_ids, col):
        if msg not in map_msg_thread:
            map_msg_thread[msg] = c

    index = df2.index.tolist()
    col_to_add = []
    for i in index:
        col_to_add.append(map_msg_thread[i])
    df2[col_name] = col_to_add
    return df2

In [81]:
messages = map_col_to_messages('thread_id')
messages = map_col_to_messages('timestamp')
messages= map_col_to_messages('sender_id')
messages = map_col_to_messages('parent_predict')
messages = map_col_to_messages('parent_prob')
messages = map_col_to_messages('response_time')
messages = map_col_to_messages('sender_gender')

In [82]:
clusters = pickle.load(open('clusters_kmeans.p', 'rb'))
messages['cluster'] = clusters

In [53]:
messages.head()

,sentiment,concrete,request_prob,polite_prob,question,inform,conventional,request_action,request_info,msg_size,adv,articles,aux_verbs,imp_prons,female_recipients,female_prons,male_recipients,male_prons,subordinates,superiors,unique_tokens,gender_unknown,power_unknown,parent_prob,per_prons,preps,quants,sub_conjs,thread_id,timestamp,sender_id,parent_predict,response_time,sender_gender,cluster
msg_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3,1.500000,2.602371,0.383466,0.517987,0.0,0.501998,0.004534,0.010361,0.009096,3.0,0.500000,0.250000,0.750000,0.0,0.0,0.0,5.0,0.000000,0.0,0.0,15.000000,7.0,12.0,NaN,2.500000,1.500000,0.250000,0.250000,13809,2001-05-04 10:58:07,376,0,NaN,I,0
5,1.333333,3.134820,0.308676,0.515260,0.0,0.363733,0.244143,0.008651,0.010642,5.0,0.166667,0.500000,0.166667,0.0,0.0,0.0,5.0,0.166667,0.0,0.0,11.500000,6.0,11.0,NaN,0.666667,1.500000,0.500000,0.166667,37634,2001-05-03 08:58:09,376,0,NaN,I,15
6,1.555556,2.397724,0.175903,0.531130,0.0,0.615570,0.003230,0.004679,0.004471,8.0,0.666667,1.666667,0.444444,0.0,0.0,0.0,5.0,0.000000,0.0,0.0,16.555556,6.0,11.0,NaN,0.222222,1.555556,0.555556,0.222222,37634,2001-05-03 07:24:24,376,0,NaN,I,1
37,1.333333,2.514151,0.230190,0.492014,0.0,0.585759,0.002017,0.022334,0.003986,5.0,0.666667,2.000000,0.166667,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,16.833333,2.0,2.0,NaN,0.333333,1.500000,1.500000,0.500000,17747,2001-10-30 21:03:49,961,0,NaN,I,1
39,1.347826,2.384991,0.270916,0.539568,0.0,0.609613,0.001993,0.002383,0.013909,22.0,0.434783,1.391304,0.826087,0.0,0.0,0.0,5.0,0.000000,0.0,0.0,20.217391,10.0,15.0,NaN,1.782609,2.086957,1.043478,0.347826,1944,2001-06-18 17:49:45,376,0,NaN,I,1


In [83]:
messages['timestamp'] = pd.to_datetime(messages['timestamp'])
messages['cluster'] = messages['cluster'].astype(str)

In [85]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71246 entries, 3 to 1367876
Data columns (total 35 columns):
sentiment            71246 non-null float64
concrete             71246 non-null float64
request_prob         71182 non-null float64
polite_prob          69286 non-null float64
question             71246 non-null float64
inform               71246 non-null float64
conventional         71246 non-null float64
request_action       71246 non-null float64
request_info         71246 non-null float64
msg_size             71246 non-null float64
adv                  71246 non-null float64
articles             71246 non-null float64
aux_verbs            71246 non-null float64
imp_prons            71246 non-null float64
female_recipients    71246 non-null float64
female_prons         71246 non-null float64
male_recipients      71246 non-null float64
male_prons           71246 non-null float64
subordinates         71246 non-null float64
superiors            71246 non-null float64
unique_to

In [60]:
clusters = messages.groupby('cluster')
clusters.mean()

,sentiment,concrete,request_prob,polite_prob,question,inform,conventional,request_action,request_info,msg_size,adv,articles,aux_verbs,imp_prons,female_recipients,female_prons,male_recipients,male_prons,subordinates,superiors,unique_tokens,gender_unknown,power_unknown,parent_prob,per_prons,preps,quants,sub_conjs
cluster,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.721346,2.584741,0.338893,0.537204,0.017890,0.500695,0.132543,0.006027,0.035793,5.217993,0.490678,0.786930,0.524454,0.000273,0.966273,0.005556,1.374140,0.012837,0.107762,0.066557,14.533666,5.232276,7.398370,0.969411,1.281184,1.080873,0.522195,0.332579
1,1.300962,2.550397,0.186857,0.506856,0.011770,0.567661,0.040208,0.004634,0.015161,10.860832,0.509959,1.632349,0.488380,0.000379,1.137287,0.003321,1.990095,0.015743,0.106194,0.104313,21.947722,8.891424,11.808300,0.970254,0.703022,1.999359,0.990736,0.365111
10,1.726985,2.630397,0.290584,0.498587,0.051341,0.498333,0.127510,0.005525,0.049665,4.700000,0.553082,0.797060,0.580408,0.000287,1.342308,0.451905,1.110577,0.027072,0.165385,0.102885,15.722773,5.449038,7.633654,0.939090,1.632041,1.217816,0.533568,0.338116
11,1.695360,2.570445,0.331392,0.508836,0.004266,0.458666,0.120500,0.007667,0.021548,5.976744,0.313601,0.873627,0.409213,0.000000,6.868217,0.005134,8.093023,0.006796,29.372093,0.325581,14.865686,62.093023,47.356589,NaN,0.831199,1.192538,0.953500,0.250167
12,1.524158,2.789036,0.248151,0.505182,0.008358,0.445999,0.118297,0.006831,0.013844,9.530864,0.257127,0.925072,0.449712,0.000000,42.382716,0.006901,85.765432,0.008959,0.074074,0.024691,17.229436,377.185185,505.234568,NaN,0.686742,1.298157,0.907522,0.233669
13,1.555936,2.535282,0.635517,0.521812,0.013786,0.389258,0.142969,0.087480,0.058298,1.395498,0.245977,0.781264,0.218866,0.000000,1.662379,0.007878,2.083601,0.005282,0.183280,0.151125,15.156486,10.755627,14.167203,0.867590,0.803065,0.903046,0.561837,0.310020
14,1.678688,2.715031,0.272326,0.485279,0.049603,0.433276,0.145287,0.005329,0.043524,2.882353,0.550006,0.677684,0.436420,0.407219,0.926471,0.000000,1.897059,0.111957,0.264706,0.088235,14.070022,5.661765,8.132353,0.789036,0.828951,0.997934,0.578163,0.250444
15,1.437728,2.648076,0.261866,0.471903,0.010532,0.392702,0.103674,0.005925,0.018224,6.462076,0.199853,0.475913,0.173778,0.000096,0.957442,0.001661,1.391788,0.004527,0.084189,0.050613,12.056367,5.855931,8.070359,0.967631,0.386647,1.033681,0.648322,0.120237
16,1.457328,2.714123,0.153374,0.466899,0.022483,0.396897,0.065624,0.004258,0.020424,187.065502,0.419258,1.163582,0.343015,0.001114,1.737991,0.008626,2.912664,0.049309,0.000000,0.056769,18.040168,10.454148,15.048035,NaN,0.513400,1.472409,0.798481,0.268119


### Get recipients

In [63]:
path = '/Users/yuwenwu/insight/cultivate/data/processed/'
chats_raw = pd.read_csv(path + 'enron_case_study_chats.csv')

In [93]:
thread_id_map = pickle.load(open('thread_id_map.p', 'rb'))
sender_id_map = pickle.load(open('sender_id_map.p', 'rb'))

In [75]:
chats_raw.head()

,chat_id,F,I,M,members
0,--5GMGaUK75qgWuZy8Wf7A,1,13,1,301LjU_cYSyatHsS_QqARA#70_DUyZ72y5ORTmgypWxcQ#...
1,--8Lsn8rO2cndTQFej3oBw,0,1,2,YxvO3GBldl05tNEc1lcTVg#fVYermj1y4sKH-7CDidb2w
2,--Isa3h8TZJECo1YBauDLQ,1,3,1,2Kc6YdWK6kU5izoj7mX9GA#8KASkpaF_9z0ZphvE0zVbg#...
3,--MDV8_WVhuXLl376p1vTg,0,36,23,1xxh4Qt9ExQiRgIg7RCRzg#3WqW9UQgAF4Ok-AbQCfteA#...
4,--N1MDh0p7vDAl1TcRkGog,1,2,1,J3w81Jm3_kW7NIa-1BOUlQ#cRBzhBG1MLBfgXxzCAoK2w#...


In [86]:
chats_raw['chat_id'] = chats_raw['chat_id'].map(thread_id_map)

In [90]:
threads = chats_raw.dropna()
threads.head()

,chat_id,F,I,M,members
1,1320,0,1,2,YxvO3GBldl05tNEc1lcTVg#fVYermj1y4sKH-7CDidb2w
2,1504,1,3,1,2Kc6YdWK6kU5izoj7mX9GA#8KASkpaF_9z0ZphvE0zVbg#...
3,1310,0,36,23,1xxh4Qt9ExQiRgIg7RCRzg#3WqW9UQgAF4Ok-AbQCfteA#...
4,1333,1,2,1,J3w81Jm3_kW7NIa-1BOUlQ#cRBzhBG1MLBfgXxzCAoK2w#...
5,1475,0,8,4,2Kc6YdWK6kU5izoj7mX9GA#EZJt9RpnAncwivDwoZ-B7A#...


In [91]:
len(threads)
messages['thread_id'].nunique()

42232

42232

In [102]:
members = threads['members'].tolist()
indiv_members = []
new_id_num = len(sender_id_map)
for member in members:
    #list of original member names
    list_members = member.split('#')
    new_member_ids = []
    for m in list_members:
        if m in sender_id_map:
            m = sender_id_map[m]
        else:
            m = new_id_num
            new_id_num += 1
        new_member_ids.append(m)
    indiv_members.append(new_member_ids)

indivs = [item for sublist in indiv_members for item in sublist]
indivs = list(set(indivs))

In [106]:
len(indiv_members) == len(threads)

True

In [113]:
len(indivs)
messages['sender_id'].nunique()

303430

8093

In [115]:
all_people = list(set(indivs + messages['sender_id'].unique().tolist()))
len(all_people)

303430

In [110]:
#key = individual member
#value = chat
msg_recipients = {}
chat_ids = threads['chat_id'].tolist()
for chat, member in zip(chat_ids, indiv_members):
    for m in member:
        msg_recipients[m] = chat

In [147]:
mems = []
thrd = []
for k, v in msg_recipients.items():
    mems.append(k)
    thrd.append(v)

for_neo_receivers = pd.DataFrame({
    'thread_id':thrd,
    'receiver_id':mems
})

In [151]:
for_neo_receivers = for_neo_receivers[for_neo_receivers.columns[::-1]]

In [152]:
for_neo_receivers.head()

,thread_id,receiver_id
0,39538,522
1,42074,307
2,42113,833
3,42130,28
4,42123,18


In [171]:
#messages = messages.reset_index()
for_neo_message = messages[['msg_id', 'thread_id', 'timestamp', 'sender_id']]

In [172]:
for_neo_message.head()

,msg_id,thread_id,timestamp,sender_id
0,3,13809,2001-05-04 10:58:07,376
1,5,37634,2001-05-03 08:58:09,376
2,6,37634,2001-05-03 07:24:24,376
3,37,17747,2001-10-30 21:03:49,961
4,39,1944,2001-06-18 17:49:45,376


In [178]:
sorted_msg_date = for_neo_message.sort_values('timestamp')

In [192]:
sorted_msg_date.head()

,msg_id,thread_id,timestamp,sender_id
40014,781240,11612,1979-12-31 19:00:00,2
39350,779243,33435,1979-12-31 19:00:00,2
39349,779242,33435,1979-12-31 19:00:00,2
39344,779197,37632,1979-12-31 19:00:00,2
39323,779057,29345,1979-12-31 19:00:00,2


In [191]:
t = '309'
sorted_msg_date[sorted_msg_date['thread_id'] == t]
for_neo_receivers[for_neo_receivers['thread_id'] == t]

,msg_id,thread_id,timestamp,sender_id
9654,70052,309,2001-07-16 10:50:39,266


,thread_id,receiver_id
7419,309,266
7420,309,13031


In [173]:
pickle.dump(for_neo_message, open('msg_sender_info.p', 'wb'))
pickle.dump(for_neo_receivers, open('msg_receiver_info.p', 'wb'))

In [169]:
for_neo_message.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71246 entries, 0 to 71245
Data columns (total 5 columns):
msg_id       71246 non-null int64
thread_id    71246 non-null object
timestamp    71246 non-null datetime64[ns]
sender_id    71246 non-null object
cluster      71246 non-null object
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 2.7+ MB


In [170]:
for_neo_message.to_csv('for_neo_message.csv', index = False)
for_neo_receivers.to_csv('for_neo_receivers.csv', index = False)

LOAD CSV FROM 'file:///home/nate/Downloads/file.csv' AS line
MERGE (n:A {number : line[0]})
WITH line, n
MERGE (m:B {ID : line[1]})
WITH m,n
MERGE (n)-[:LIKES]->(m);

## Neo4j

In [61]:
from neo4jrestclient.client import GraphDatabase

In [62]:
db = GraphDatabase("http://localhost:7474", username="neo4j", password="mypassword")

In [ ]:
# Create some nodes with labels
#user = db.labels.create("User")
# u1 = db.nodes.create(name="Marco")
# user.add(u1)
person = db.labels.create('Person')

def create_nodes(list_):
    for i range(len(list_)):
        

In [116]:
from py2neo import Graph, Path

In [ ]:
# graph = Graph()

# tx = graph.cypher.begin()
# for name in ["Alice", "Bob", "Carol"]:
#     tx.append("CREATE (person:Person {name:{name}}) RETURN person", name=name)
# alice, bob, carol = [result.one for result in tx.commit()]

# friends = Path(alice, "KNOWS", bob, "KNOWS", carol)
# graph.create(friends)